# Sample 02: OpenAI SDK ಏಕೀಕರಣ

ಈ ನೋಟ್ಬುಕ್ OpenAI Python SDK ನೊಂದಿಗೆ ಉನ್ನತ ಮಟ್ಟದ ಏಕೀಕರಣವನ್ನು ಪ್ರದರ್ಶಿಸುತ್ತದೆ, ಇದು Microsoft Foundry Local ಮತ್ತು Azure OpenAI ಎರಡನ್ನೂ ಸ್ಟ್ರೀಮಿಂಗ್ ಪ್ರತಿಕ್ರಿಯೆಗಳೊಂದಿಗೆ ಮತ್ತು ಸರಿಯಾದ ದೋಷ ನಿರ್ವಹಣೆಯೊಂದಿಗೆ ಬೆಂಬಲಿಸುತ್ತದೆ.

## ಅವಲೋಕನ

ಈ ಮಾದರಿ ತೋರಿಸುತ್ತದೆ:
- Foundry Local ಮತ್ತು Azure OpenAI ನಡುವೆ ನಿರಂತರ ಸ್ವಿಚಿಂಗ್
- ಉತ್ತಮ ಬಳಕೆದಾರ ಅನುಭವಕ್ಕಾಗಿ ಸ್ಟ್ರೀಮಿಂಗ್ ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆಗಳು
- FoundryLocalManager SDK ಯ ಸರಿಯಾದ ಬಳಕೆ
- ಬಲವಾದ ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ಬ್ಯಾಕ್ಅಪ್ ಯಂತ್ರಗಳು
- ಉತ್ಪಾದನೆಗೆ ಸಿದ್ಧವಾದ ಕೋಡ್ ಮಾದರಿಗಳು


## ಪೂರ್ವಾಪೇಕ್ಷೆಗಳು

- **Foundry Local**: ಸ್ಥಾಪಿಸಲಾಗಿದೆ ಮತ್ತು ಚಾಲನೆಯಲ್ಲಿದೆ (ಸ್ಥಳೀಯ ನಿರ್ಣಯಕ್ಕಾಗಿ)
- **Python**: 3.8 ಅಥವಾ ನಂತರದ ಆವೃತ್ತಿ OpenAI SDK ಜೊತೆಗೆ
- **Azure OpenAI**: ಮಾನ್ಯ ಎಂಡ್‌ಪಾಯಿಂಟ್ ಮತ್ತು API ಕೀ (ಮೇಘ ನಿರ್ಣಯಕ್ಕಾಗಿ)

### ಅವಲಂಬನೆಗಳನ್ನು ಸ್ಥಾಪಿಸಿ


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## ಗ್ರಂಥಾಲಯಗಳನ್ನು ಆಮದುಮಾಡಿ ಮತ್ತು ಸೆಟಪ್ ಮಾಡಿ


In [ ]:
import os
import sys
from openai import OpenAI
import time
from typing import Tuple

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, manual configuration will be used")

## ಸಂರಚನಾ ಆಯ್ಕೆಗಳು

ಸಂಬಂಧಿತ ಪರಿಸರ ಚರಗಳನ್ನು ಹೊಂದಿಸುವ ಮೂಲಕ Azure OpenAI (ಮೇಘ) ಅಥವಾ Foundry Local (ಉಪಕರಣದಲ್ಲಿ) ನಡುವೆ ಆಯ್ಕೆಮಾಡಿ.


### ಆಯ್ಕೆ 1: ಅಜೂರ್ ಓಪನ್‌ಎಐ ಸಂರಚನೆ

ನಿಮ್ಮ ಅಜೂರ್ ಓಪನ್‌ಎಐ ಪ್ರಮಾಣಪತ್ರಗಳನ್ನು ಅನಕಮೆಂಟ್ ಮಾಡಿ ಮತ್ತು ಸೆಟ್ ಮಾಡಿ:


In [ ]:
# Azure OpenAI Configuration
# Uncomment and set your actual values

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"  # e.g., "gpt-4"

print("Azure OpenAI configuration ready (if credentials are set)")

### ಆಯ್ಕೆ 2: ಫೌಂಡ್ರಿ ಸ್ಥಳೀಯ ಸಂರಚನೆ

ಸ್ಥಳೀಯ ನಿರ್ಣಯಕ್ಕಾಗಿ ಡೀಫಾಲ್ಟ್ ಸೆಟ್ಟಿಂಗ್‌ಗಳು:


In [ ]:
# Foundry Local Configuration (default)
FOUNDRY_MODEL = "phi-4-mini"  # Change to your preferred model
FOUNDRY_BASE_URL = "http://localhost:51211"
FOUNDRY_API_KEY = ""  # Usually empty for local

print(f"Foundry Local configuration ready with model: {FOUNDRY_MODEL}")

## ಕ್ಲೈಂಟ್ ಫ್ಯಾಕ್ಟರಿ ಫಂಕ್ಷನ್ಗಳು

ನಿಮ್ಮ ಸಂರಚನೆಯ ಆಧಾರದ ಮೇಲೆ ಸೂಕ್ತವಾದ OpenAI ಕ್ಲೈಂಟ್ ಅನ್ನು ಈ ಫಂಕ್ಷನ್ಗಳು ರಚಿಸುತ್ತವೆ:


In [ ]:
def create_azure_client() -> Tuple[OpenAI, str]:
    """Create Azure OpenAI client."""
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if not azure_endpoint or not azure_api_key:
        raise ValueError("Azure OpenAI endpoint and API key are required")
    
    model = os.environ.get("MODEL", "your-deployment-name")
    client = OpenAI(
        base_url=f"{azure_endpoint}/openai",
        api_key=azure_api_key,
        default_query={"api-version": azure_api_version},
    )
    
    print(f"🌐 Azure OpenAI client created with model: {model}")
    return client, model


def create_foundry_client() -> Tuple[OpenAI, str]:
    """Create Foundry Local client with SDK management."""
    alias = FOUNDRY_MODEL
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            print(f"🔄 Initializing Foundry Local with model: {alias}...")
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            
            print(f"✅ Foundry Local SDK initialized")
            print(f"   Endpoint: {manager.endpoint}")
            print(f"   Model: {model_info.id}")
            return client, model_info.id
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{FOUNDRY_BASE_URL}/v1",
        api_key=FOUNDRY_API_KEY
    )
    
    print(f"🔧 Manual Foundry Local configuration")
    print(f"   Endpoint: {FOUNDRY_BASE_URL}/v1")
    print(f"   Model: {alias}")
    return client, alias

## ಕ್ಲೈಂಟ್ ಪ್ರಾರಂಭಿಸಿ

ಇದು ಸ್ವಯಂಚಾಲಿತವಾಗಿ Azure OpenAI ಅಥವಾ Foundry Local ಅನ್ನು ಬಳಸಬೇಕೇ ಎಂದು ಪತ್ತೆಹಚ್ಚುತ್ತದೆ:


In [ ]:
def initialize_client() -> Tuple[OpenAI, str, str]:
    """Initialize the appropriate OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if azure_endpoint and azure_api_key:
        print("🌐 Azure OpenAI configuration detected")
        try:
            client, model = create_azure_client()
            return client, model, "azure"
        except Exception as e:
            print(f"❌ Azure OpenAI initialization failed: {e}")
            print("🔄 Falling back to Foundry Local...")
    
    # Use Foundry Local
    print("🏠 Using Foundry Local configuration")
    try:
        client, model = create_foundry_client()
        return client, model, "foundry"
    except Exception as e:
        print(f"❌ Foundry Local initialization failed: {e}")
        raise

# Initialize the client
print("Initializing OpenAI client...")
print("=" * 50)
client, model, provider = initialize_client()
print("=" * 50)
print(f"✅ Initialization complete! Using {provider} with model: {model}")

## ಮೂಲ ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆ

ಸರಳ ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆಯನ್ನು ಪರೀಕ್ಷಿಸಿ:


In [ ]:
def simple_chat(prompt: str, max_tokens: int = 150) -> str:
    """Send a simple chat message and get response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test basic chat
test_prompt = "Say hello from the SDK quickstart and explain what you are in one sentence."

print(f"👤 User: {test_prompt}")
print("\n🤖 Assistant:")
response = simple_chat(test_prompt)
print(response)

## ಸ್ಟ್ರೀಮಿಂಗ್ ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆ

ಉತ್ತಮ ಬಳಕೆದಾರ ಅನುಭವಕ್ಕಾಗಿ ಸ್ಟ್ರೀಮಿಂಗ್ ಪ್ರತಿಕ್ರಿಯೆಗಳನ್ನು ಪ್ರದರ್ಶಿಸಿ:


In [ ]:
def streaming_chat(prompt: str, max_tokens: int = 300) -> str:
    """Send a chat message with streaming response."""
    try:
        print("🤖 Assistant (streaming):")
        
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        error_msg = f"Error: {e}"
        print(error_msg)
        return error_msg

# Test streaming chat
streaming_prompt = "Explain the key benefits of using Microsoft Foundry Local for AI development. Include aspects like privacy, performance, and cost."

print(f"👤 User: {streaming_prompt}\n")
streaming_response = streaming_chat(streaming_prompt)

## ಬಹು-ತಿರುವು ಸಂಭಾಷಣೆ

ಸಂಭಾಷಣೆ ಸಂದರ್ಭವನ್ನು ಕಾಪಾಡುವುದನ್ನು ಪ್ರದರ್ಶಿಸಿ:


In [ ]:
class ConversationManager:
    """Manages multi-turn conversations with context."""
    
    def __init__(self, system_prompt: str = None):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send_message(self, user_message: str, max_tokens: int = 200) -> str:
        """Send a message and get response while maintaining context."""
        # Add user message to conversation
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=self.messages,
                max_tokens=max_tokens
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add assistant response to conversation
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_length(self) -> int:
        """Get the number of messages in the conversation."""
        return len(self.messages)

# Create conversation manager with system prompt
system_prompt = "You are a helpful AI assistant specialized in explaining AI and machine learning concepts. Be concise but informative."
conversation = ConversationManager(system_prompt)

# Multi-turn conversation example
conversation_turns = [
    "What is the difference between AI inference on-device vs in the cloud?",
    "Which approach is better for privacy?",
    "What about performance and latency considerations?"
]

for i, turn in enumerate(conversation_turns, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {turn}")
    
    response = conversation.send_message(turn)
    print(f"\n🤖 Assistant: {response}")

print(f"\n📊 Conversation summary: {conversation.get_conversation_length()} messages total")

## ಕಾರ್ಯಕ್ಷಮತೆ ಹೋಲಿಕೆ

ವಿಭಿನ್ನ ಸಂದರ್ಭಗಳಿಗಾಗಿ ಪ್ರತಿಕ್ರಿಯೆ ಸಮಯಗಳನ್ನು ಹೋಲಿಸಿ:


In [ ]:
def benchmark_response_time(prompt: str, iterations: int = 3) -> dict:
    """Benchmark response time for a given prompt."""
    times = []
    responses = []
    
    for i in range(iterations):
        start_time = time.time()
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50  # Keep responses short for timing
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            times.append(response_time)
            responses.append(response.choices[0].message.content)
            
        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        return {
            "average_time": avg_time,
            "min_time": min_time,
            "max_time": max_time,
            "all_times": times,
            "sample_response": responses[0] if responses else None
        }
    
    return {"error": "No successful responses"}

# Benchmark different types of prompts
benchmark_prompts = [
    "What is AI?",
    "Explain machine learning in simple terms.",
    "List 3 benefits of edge computing."
]

print(f"⏱️  Performance Benchmark ({provider} - {model})")
print("=" * 60)

for prompt in benchmark_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    results = benchmark_response_time(prompt)
    
    if "error" not in results:
        print(f"   ⏰ Average time: {results['average_time']:.2f}s")
        print(f"   ⚡ Fastest: {results['min_time']:.2f}s")
        print(f"   🐌 Slowest: {results['max_time']:.2f}s")
        print(f"   📄 Sample response: {results['sample_response'][:100]}...")
    else:
        print(f"   ❌ {results['error']}")

## ಉನ್ನತ ಸಂರಚನೆ ಮತ್ತು ದೋಷ ನಿರ್ವಹಣೆ

ವಿಭಿನ್ನ ಪರಿಮಾಣಗಳು ಮತ್ತು ದೋಷ ಸಂದರ್ಭಗಳನ್ನು ಪರೀಕ್ಷಿಸಿ:


In [ ]:
def test_different_parameters():
    """Test chat completions with different parameters."""
    prompt = "Write a creative short story about AI."
    
    # Test different temperature values
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=temp
            )
            
            print(f"Response: {response.choices[0].message.content[:150]}...")
            
        except Exception as e:
            print(f"Error with temperature {temp}: {e}")

test_different_parameters()

## ಸೇವೆ ಆರೋಗ್ಯ ಪರಿಶೀಲನೆ

ವಿಸ್ತೃತ ಸೇವೆ ಆರೋಗ್ಯ ಮತ್ತು ಸಾಮರ್ಥ್ಯ ಪರಿಶೀಲನೆ:


In [ ]:
def comprehensive_health_check():
    """Perform comprehensive health check of the service."""
    print("🏥 Comprehensive Health Check")
    print("=" * 50)
    
    # 1. Check model listing
    try:
        models_response = client.models.list()
        available_models = [m.id for m in models_response.data]
        print(f"✅ Model listing: SUCCESS")
        print(f"   📋 Available models: {available_models}")
        
        if model in available_models:
            print(f"   ✅ Current model '{model}' is available")
        else:
            print(f"   ⚠️ Current model '{model}' not found in available models")
    except Exception as e:
        print(f"❌ Model listing: FAILED - {e}")
    
    # 2. Test basic completion
    try:
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Say 'Health check successful'"}],
            max_tokens=10
        )
        print(f"✅ Basic completion: SUCCESS")
        print(f"   💬 Response: {test_response.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Basic completion: FAILED - {e}")
    
    # 3. Test streaming
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Count to 3"}],
            max_tokens=20,
            stream=True
        )
        
        stream_content = ""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices[0].delta.content:
                stream_content += chunk.choices[0].delta.content
                chunk_count += 1
        
        print(f"✅ Streaming: SUCCESS")
        print(f"   📦 Chunks received: {chunk_count}")
        print(f"   💬 Streamed content: {stream_content.strip()}")
    except Exception as e:
        print(f"❌ Streaming: FAILED - {e}")
    
    # 4. Provider-specific information
    print(f"\n📊 Configuration Summary:")
    print(f"   🏢 Provider: {provider}")
    print(f"   🤖 Model: {model}")
    if provider == "foundry":
        print(f"   🏠 Foundry SDK Available: {FOUNDRY_SDK_AVAILABLE}")
        print(f"   🔗 Base URL: {FOUNDRY_BASE_URL}")
    elif provider == "azure":
        print(f"   🌐 Azure Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
        print(f"   🔑 API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

comprehensive_health_check()

## ಇಂಟರಾಕ್ಟಿವ್ ಪರೀಕ್ಷೆ

ನಿಮ್ಮ ಸ್ವಂತ ಪ್ರಾಂಪ್ಟ್‌ಗಳನ್ನು ಇಂಟರಾಕ್ಟಿವ್ ಆಗಿ ಪರೀಕ್ಷಿಸಲು ಈ ಸೆಲ್ ಅನ್ನು ಬಳಸಿ:


In [ ]:
# Interactive testing - modify the prompt below
custom_prompt = "Explain the concept of 'edge AI' and why it's becoming important."
use_streaming = True  # Set to False for regular completion

print(f"👤 Custom Prompt: {custom_prompt}\n")

if use_streaming:
    custom_response = streaming_chat(custom_prompt, max_tokens=250)
else:
    custom_response = simple_chat(custom_prompt, max_tokens=250)
    print(f"🤖 Assistant: {custom_response}")

## ಸಾರಾಂಶ ಮತ್ತು ಮುಂದಿನ ಹಂತಗಳು

ಈ ನೋಟ್ಬುಕ್ ಮುಂದಿನ OpenAI SDK ಏಕೀಕರಣವನ್ನು ತೋರಿಸಿತು:

### ✅ ಪ್ರಮುಖ ವೈಶಿಷ್ಟ್ಯಗಳು ಒಳಗೊಂಡಿವೆ

1. **ಬಹು-ಪ್ರದಾತ ಬೆಂಬಲ**: Azure OpenAI ಮತ್ತು Foundry Local ನಡುವೆ ಸುಗಮ ಸ್ವಿಚಿಂಗ್  
2. **ಸ್ಟ್ರೀಮಿಂಗ್ ಪ್ರತಿಕ್ರಿಯೆಗಳು**: ಉತ್ತಮ ಬಳಕೆದಾರ ಅನುಭವಕ್ಕಾಗಿ ರಿಯಲ್-ಟೈಮ್ ಟೋಕನ್ ಉತ್ಪಾದನೆ  
3. **ಸಂವಾದ ನಿರ್ವಹಣೆ**: ಸಾಂದರ್ಭಿಕ ಬಹು-ತಿರುವು ಸಂವಾದಗಳು  
4. **ಕಾರ್ಯಕ್ಷಮತೆ ಮೌಲ್ಯಮಾಪನ**: ಪ್ರತಿಕ್ರಿಯೆ ಸಮಯದ ಅಳೆಯುವಿಕೆ ಮತ್ತು ವಿಶ್ಲೇಷಣೆ  
5. **ಸಮಗ್ರ ಆರೋಗ್ಯ ಪರಿಶೀಲನೆಗಳು**: ಸೇವೆ ಮಾನ್ಯತೆ ಮತ್ತು ರೋಗನಿರ್ಣಯ  
6. **ದೋಷ ನಿರ್ವಹಣೆ**: ಬಲವಾದ ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ಬ್ಯಾಕ್ಅಪ್ ವ್ಯವಸ್ಥೆಗಳು  

### 🏆 Foundry Local ಮತ್ತು Azure OpenAI

| ಅಂಶ | Foundry Local | Azure OpenAI |
|--------|---------------|---------------|
| **ಗೌಪ್ಯತೆ** | ✅ ಡೇಟಾ ಸ್ಥಳೀಯವಾಗಿಯೇ ಉಳಿಯುತ್ತದೆ | ⚠️ ಡೇಟಾ ಕ್ಲೌಡ್‌ಗೆ ಕಳುಹಿಸಲಾಗುತ್ತದೆ |
| **ವಿಲಂಬ** | ✅ ಕಡಿಮೆ (ಸ್ಥಳೀಯ ನಿರ್ಣಯ) | ⚠️ ಹೆಚ್ಚು (ನೆಟ್‌ವರ್ಕ್ ಅವಲಂಬನೆ) |
| **ಖರ್ಚು** | ✅ ಉಚಿತ (ಹಾರ್ಡ್‌ವೇರ್ ನಂತರ) | 💰 ಪ್ರತಿ ಟೋಕನ್‌ಗೆ ಪಾವತಿ |
| **ಆಫ್‌ಲೈನ್** | ✅ ಆಫ್‌ಲೈನ್‌ನಲ್ಲಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ | ❌ ಇಂಟರ್ನೆಟ್ ಅಗತ್ಯವಿದೆ |
| **ಮಾದರಿ ವೈವಿಧ್ಯತೆ** | ⚠️ ಸೀಮಿತ ಆಯ್ಕೆ | ✅ ಸಂಪೂರ್ಣ ಮಾದರಿ ಪ್ರವೇಶ |
| **ಸ್ಕೇಲಿಂಗ್** | ⚠️ ಹಾರ್ಡ್‌ವೇರ್ ಅವಲಂಬಿತ | ✅ ಅನಿಯಮಿತ ಸ್ಕೇಲಿಂಗ್ |

### 🚀 ಮುಂದಿನ ಹಂತಗಳು

- **ಸ್ಯಾಂಪಲ್ 04**: Chainlit ಚಾಟ್ ಅಪ್ಲಿಕೇಶನ್ ನಿರ್ಮಾಣ  
- **ಸ್ಯಾಂಪಲ್ 05**: ಬಹು-ಏಜೆಂಟ್ ಸಂಯೋಜನಾ ವ್ಯವಸ್ಥೆಗಳು  
- **ಸ್ಯಾಂಪಲ್ 06**: ಬುದ್ಧಿವಂತ ಮಾದರಿ ಮಾರ್ಗದರ್ಶನ  
- **ಉತ್ಪಾದನಾ ನಿಯೋಜನೆ**: ಸ್ಕೇಲಿಂಗ್ ಮತ್ತು ಮೇಲ್ವಿಚಾರಣೆಯ ಪರಿಗಣನೆಗಳು  

### 💡 ಉತ್ತಮ ಅಭ್ಯಾಸಗಳು

1. **ಪ್ರದಾತಗಳ ನಡುವೆ ಯಾವಾಗಲೂ ಬ್ಯಾಕ್ಅಪ್ ವ್ಯವಸ್ಥೆಗಳನ್ನು ಜಾರಿಗೆ ತರುವಿರಿ**  
2. **ದೀರ್ಘ ಪ್ರತಿಕ್ರಿಯೆಗಳಿಗೆ ಸ್ಟ್ರೀಮಿಂಗ್ ಬಳಸಿ** ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ಸುಧಾರಿಸಲು  
3. **ಉತ್ಪಾದನಾ ಅಪ್ಲಿಕೇಶನ್‌ಗಳಿಗೆ ಸರಿಯಾದ ದೋಷ ನಿರ್ವಹಣೆಯನ್ನು ಜಾರಿಗೆ ತರುವಿರಿ**  
4. **ಪ್ರತಿಕ್ರಿಯೆ ಸಮಯ ಮತ್ತು ಖರ್ಚುಗಳನ್ನು ನಿಗಾ ವಹಿಸಿ** ವಿಭಿನ್ನ ಪ್ರದಾತರಿಗಾಗಿ  
5. **ನಿಮ್ಮ ನಿರ್ದಿಷ್ಟ ಅಗತ್ಯಗಳ ಆಧಾರದ ಮೇಲೆ ಸರಿಯಾದ ಪ್ರದಾತರನ್ನು ಆಯ್ಕೆಮಾಡಿ**


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
